In [ ]:
import pandas as pd
df=pd.read_csv("IndianFinancialNews.csv")
df.head()

,Unnamed: 0,Date,Title,Description
0,0,"May 26, 2020, Tuesday","ATMs to become virtual bank branches, accept d...","Close to 14.6 per cent (or 35,000) of the 240,..."
1,1,"May 26, 2020, Tuesday",IDFC First Bank seniors to forgo 65% of bonus ...,"V Vaidyanathan, managing director and chief ex..."
2,2,"May 25, 2020, Monday","Huge scam in YES Bank for many years, says Enf...",Rana Kapoor's wife also charged with abetting ...
3,3,"May 24, 2020, Sunday","Bank of Maharashtra sanctioned Rs 2,789 cr in ...",The bank said it was now gearing up to extend ...
4,4,"May 23, 2020, Saturday",DCB Bank's profit before tax declines 37.6% to...,Net profit for the financial year ended March ...


In [ ]:
df.isnull().sum()

Unnamed: 0       0
Date             0
Title            0
Description    710
dtype: int64

In [ ]:
# Identify rows with missing values
rows_with_missing_values = df[df.isnull().any(axis=1)]

# Display these rows
print("Rows with missing values:")
print(rows_with_missing_values)

Rows with missing values:
       Unnamed: 0                        Date  \
34261       34261  October 11, 2008, Saturday   
34263       34263  October 11, 2008, Saturday   
35019       35019       July 25, 2008, Friday   
35020       35020       July 25, 2008, Friday   
35021       35021       July 25, 2008, Friday   
...           ...                         ...   
35805       35805   April 23, 2008, Wednesday   
35825       35825    April 19, 2008, Saturday   
35830       35830      April 18, 2008, Friday   
35838       35838    April 17, 2008, Thursday   
35848       35848   April 16, 2008, Wednesday   

                                                   Title Description  
34261            Finance unit sharpens GE Q3 profit fall         NaN  
34263  IMF to use emergency loan programme to tackle ...         NaN  
35019                         PNB to exit from PNB Gilts         NaN  
35020     Karur Vysya Bank's net dips 56% at Rs 30.54 cr         NaN  
35021  Misuse of unsolicited c

In [ ]:
df.dropna(subset=['Description'], inplace=True)

In [ ]:
df.isnull().sum()

Unnamed: 0     0
Date           0
Title          0
Description    0
dtype: int64

In [ ]:
df.to_csv('cleaned_data.csv', index=False)

In [ ]:
df_1=pd.read_csv('cleaned_data.csv')
df_1.isnull().sum()

Unnamed: 0     0
Date           0
Title          0
Description    0
dtype: int64

In [ ]:
feature=df[['Description']]
feature.to_csv('Final_data.csv',index=False)

In [ ]:
df_2=pd.read_csv('Final_data.csv')
df_2.head()

,Description
0,"Close to 14.6 per cent (or 35,000) of the 240,..."
1,"V Vaidyanathan, managing director and chief ex..."
2,Rana Kapoor's wife also charged with abetting ...
3,The bank said it was now gearing up to extend ...
4,Net profit for the financial year ended March ...


In [ ]:
import numpy as np
# Select the first 100 data points
last_df = df_2.iloc[200:500]

# Subdivide into 4 groups (each with 25 data points)
groups = np.array_split(last_df, 4)

#exporting each partition
for i, group in enumerate(groups):
    group.to_csv(f"Group{i+1}.csv",index=False)

In [ ]:
#custom NER using custom Spacy
import json
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
from spacy import displacy
import random

# Load the JSON file
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)


annotation_file_1 = load_json('/content/annotations1.json')
annotation_file_2 = load_json('/content/annotations2.json')
annotation_file_3 = load_json('/content/annotations3.json')
annotation_file_4 = load_json('/content/annotations4.json')
annotation_file_5 = load_json('/content/annotations5.json')
annotation_file_6 = load_json('/content/annotations6.json')
annotation_file_7 = load_json('/content/annotations7.json')
annotation_file_8 = load_json('/content/annotations8.json')
annotation_file_9 = load_json('/content/annotations9.json')
annotation_file_10 = load_json('/content/annotations10.json')


# Convert data to spaCy format
def convert_data(data):
    training_data = []
    for text, annotations in data["annotations"]:

        entities = [(start, end, label) for start, end, label in annotations["entities"]]
        training_data.append((text, {"entities": entities}))
    return training_data

training_data = convert_data(annotation_file_1) + convert_data(annotation_file_2) + convert_data(annotation_file_3) + convert_data(annotation_file_4) + convert_data(annotation_file_5) + convert_data(annotation_file_6) + convert_data(annotation_file_7) + convert_data(annotation_file_8) + convert_data(annotation_file_9) + convert_data(annotation_file_10)

# Create a blank spaCy model
nlp = spacy.blank("en")

# Create the NER component and add it to the pipeline
ner = nlp.add_pipe("ner")

# Add new labels to the NER component
for _, annotations in training_data:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

# Disable other pipeline components (if any)
pipe_exceptions = ["ner"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# Training the NER model
with nlp.disable_pipes(*unaffected_pipes):

    optimizer = nlp.begin_training()

    for iteration in range(200):
        random.shuffle(training_data)
        losses = {}
        batches = minibatch(training_data, size=compounding(4.0, 32.0, 1.01))
        for batch in batches:
            texts, annotations = zip(*batch)
            examples = [Example.from_dict(nlp.make_doc(text), ann) for text, ann in zip(texts, annotations)]
            nlp.update(examples, drop=0.5, losses=losses)
        #print(f"Iteration {iteration + 1}, Losses: {losses}")

# Save the trained model
nlp.to_disk("custom_ner_model")

# Test the trained model
test_text = """On June 21, 2023, Acme Corp filed its quarterly earnings report with the SEC, revealing a net income of
 $15 million for Q2 2023, up from $10 million in Q2 2022. The report highlighted a 25% increase in revenue, primarily
driven by the strong performance of its new line of consumer electronics. Additionally, the company announced a dividend
payout of $0.50 per share, payable on July 15, 2023, to shareholders of record as of July 1, 2023. Acme Corp's stock
(ticker: ACME) saw a 5% rise in after-hours trading following the announcement. In contrast, bonds issued by the company
experienced a slight decrease in value, reflecting broader market concerns about rising interest rates. The CFO, Jane Doe,
reassured investors that Acme Corp remains well-positioned to navigate economic uncertainties and maintain its growth
trajectory."""

doc = nlp(test_text)
displacy.render(doc, style="ent", jupyter=True)



In [ ]:
# Test the trained model
test_text = """The company's debt-to-equity ratio stands at 2:1, indicating a healthy balance sheet.
 The price-to-earnings ratio for the tech sector is currently 20, reflecting high investor confidence.
 Analysts are closely monitoring the current ratio of companies in the retail sector, which is a key indicator
 of liquidity. The return-on-assets ratio for the banking industry has improved to 1.5%, signaling efficient asset management. The quick ratio for the manufacturing sector is below 1, raising concerns about short-term liquidity. The debt-service coverage ratio for small businesses is expected to decrease due to rising interest rates. The gross margin ratio for the pharmaceutical company improved to 40%, driven by cost-saving initiatives. The price-to-book ratio for the energy sector is 1.2, indicating undervaluation compared to industry peers. The capital adequacy ratio for banks in Europe is 12%, meeting regulatory requirements. The dividend payout ratio for the telecommunications sector is 50%, providing attractive returns for investors."""

doc = nlp(test_text)
displacy.render(doc, style="ent", jupyter=True)